In [1]:
# imprescindible
import pandas as pd
import numpy as np

# to avoid some warnings messages
import warnings
# warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

# to draw some graphs
import seaborn as sns
import matplotlib.pyplot as plt

# set seaborn and matplotlib default theme
sns.set_theme()
_sns_plotting_contex_ = sns.plotting_context()
sns.plotting_context('poster')

# set seaborn and matplotlib style to ...
# plt.style.use('classic')
sns.mpl.rcParams['axes.titlesize'] = 18
sns.mpl.rcParams['axes.labelsize'] = 14

# to use HTML codes within IPpython.display function
from IPython.display import HTML

import os


In [2]:

from sklearn.preprocessing import StandardScaler

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
import xgboost as xgb
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

# to tune hiperparameters
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import roc_auc_score
from sklearn.metrics import RocCurveDisplay

In [3]:
# to keep only 4 digits
ROUND = lambda v : round(v, 4)


In [4]:
def set_figure(row, col, suptitle=None, y=0.98, fontsize='xx-large', fontweight='extra bold') :
    u''' Activate matplot figure setting size and super title
    '''
    
    fig = plt.figure(figsize=(row, col));
    if suptitle != None :
        fig.suptitle(suptitle, y=y, 
                     verticalalignment='center', fontsize=fontsize, fontweight=fontweight);
    return fig


### Data

In [5]:
def read_blogData_train() :
    u''' Reads and prepare data from blog feedback data train set
    
    '''

    data = pd.read_csv("./data/blogData_train.csv", header=None)
    data.drop_duplicates(inplace=True)
    data.reset_index(drop=True, inplace=True)
    
    header = pd.read_csv("./data/blogData_label.csv", header=None)
    header = list(header[0])
    
    if len(header) != data.shape[1] :
        raise Exception('Los encabezados y la cantidad de características NO COINCIDE !!!')

    data.columns = header
    
    return data


In [6]:
# to_classes = lambda v : 0 if v < 30 else (1 if v < 90 else (2 if v < 150 else (3 if v < 210 else 4)))
# to_classes = lambda v : 0 if v < 30 else (1 if v < 90 else 2)
to_classes = lambda v : 0 if v < 30 else 1

---

In [7]:
import pickle
SAVE_MODELS = True

In [8]:

from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin


---


In [9]:
class SpecialScaler(BaseEstimator, TransformerMixin) :
    def __init__(self, scaler) :
        self.scaler = scaler
    
    def fit(self, X, *_) :
        self.scaler.fit(X)
        return self

    def transform(self, X, *_) :
        if isinstance(X, pd.DataFrame) :
            return pd.DataFrame(
                data=self.scaler.transform(X), 
                columns=list(X.columns))
        else:
            raise TypeError("Este Transformador solo funciona en DF de Pandas")
    
# ---    

In [10]:
class Eval_Estimator :
    u'''
    '''

    def __init__(self, name, estimator, params=None, gs_param_grid=None) :
        self.name = name
        self.estimator = estimator
        self.params = params
        # attributes used in GridSearchCV
        self.gs_param_grid = gs_param_grid
        self.gs_estimator = None

        return        
# ---

In [11]:
def get_classification_model(pkl_file, X_train, y_train) :
    u'''
    '''
# ---
    # pkl_file = './models/XGBoostClassifier.pkl'
# ---
    try :
        with open(pkl_file, 'rb') as pkl_hand :
            model = pickle.load(pkl_hand)
        print('Se cargo el modelo', model.name, 'de', pkl_file, '...')
    except :
        model = Eval_Estimator(
            name='XGBoost Classifier (pipe)', 
            estimator=Pipeline([
                ('scaler', SpecialScaler(StandardScaler())), 
                ('XGBoost', xgb.XGBClassifier(
                    eval_metric = 'auc', # 
                    gamma = 0, # (min_split_loss) minimum loss reduction
                    learning_rate = 0.0001, # (eta) step size shrinkage
                    max_delta_step = 1e6, # extremely imbalanced
                    max_depth = 20, # maximum depth of tree
                    n_estimators = 500, 
                    n_jobs = -1, # use all processors
                    objective = 'binary:logistic', # for binary classification 
                    random_state = 127, 
                    verbosity = 0
                ))
            ]) 
        )
    # ---
        print('Entrenando modelo', model.name, '...')
        # print(model.estimator.get_params())
        model.estimator.fit(X_train, y_train)
    # ---
        if SAVE_MODELS :
            with open(pkl_file, 'wb') as pkl_hand :
                pickle.dump(model, pkl_hand)
    # ---
    return model
# ---
    

In [12]:
# def evaluate_classification_model() :
#     u'''
#     '''
if True :
# ---    
    data_raw = read_blogData_train()
    # test_raw = blogData_test_read()

    X_train = data_raw.iloc[:, 0:280]
    y_train = data_raw.iloc[:, -1].copy()
    y_train = y_train.apply(to_classes)

    # X_test = test_raw.iloc[:, 0:280]
    # y_test = test_raw.iloc[:, -1].copy()

    X_train = X_train.iloc[:, 0:62].copy()
    # X_test = X_test.iloc[:, 0:62].copy()

# ---
    model = get_classification_model('./models/XGBoostClassifier_pipe.pkl', X_train, y_train)
# ---

    class Eval_TestCase :
        u'''
        '''
        def __init__(self, case, data) :
            self.case = case
            self.data = data
            self.y_test = None
            self.y_pred = None
            return    
# ---
    filepath = './data/test/'
    filelist = [os.path.join(filepath, filename) for filename in os.listdir(filepath) if os.path.isfile(os.path.join(filepath, filename))]

    caselist = []
    for filename in filelist :
        temp_raw = pd.read_csv(filename, header=None)
        temp_raw.drop_duplicates(inplace=True)
        
        pos = filename.index('2012')
        caselist.append( Eval_TestCase(filename[pos: pos+10], temp_raw) )
# ---
    results = pd.DataFrame(columns=['case', 'count', 'ROC AUC', 'y_test', 'y_pred', 'f1-score'])

    for e in caselist :
        # X = e.data.iloc[:, 0:280]
        X = e.data.iloc[:, 0:62]
        y = e.data.iloc[:, -1]
        y = y.apply(to_classes)
        
        p = model.estimator.predict(X)
        
        e.y_test = y
        e.y_pred = p
        
        d = classification_report(y_true=y, y_pred=p, output_dict=True)
        
        try :
            roc_auc = ROUND(roc_auc_score(y, p))
            f1 = ROUND(d['1']['f1-score'])
        except :
            rec_auc = -1
            f1 = np.NaN

        results = results.append(
            pd.Series(
                data=[e.case, 
                      y.shape[0], 
                      roc_auc, 
                      y, p, 
                      f1 
                     ], 
                index=results.columns), 
            ignore_index=True
        )
        
    # return results
# ---    


# comment to hide evaluate of regression model
# results = evaluate_classification_model()
display(results[['case', 'count', 'ROC AUC']].sample(5))
display(results[['ROC AUC']].describe().transpose())
display(results[['f1-score']].describe().transpose())

Entrenando modelo XGBoost Classifier (pipe) ...


,case,count,ROC AUC
13,2012.02.14,96,0.7444
11,2012.02.12,97,1.0000
48,2012.03.20,109,0.8239
0,2012.02.01,113,0.7408
14,2012.02.15,123,0.7915


,count,mean,std,min,25%,50%,75%,max
ROC AUC,60.0,0.742108,0.149739,0.491,0.6551,0.73805,0.842775,1.0


,count,mean,std,min,25%,50%,75%,max
f1-score,59.0,0.541512,0.276659,0.0,0.3818,0.5714,0.697,1.0
